<a href="https://colab.research.google.com/github/aj1365/3DGAN-ViT/blob/main/3DGAN_ViT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Importing Libraries***

In [ ]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

In [ ]:
from tensorflow import keras
import numpy as np
from keras.layers import Conv2D, Conv3D, Flatten, Dense, Reshape, BatchNormalization
from keras.layers import Dropout, Input
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score

from operator import truediv

from plotly.offline import init_notebook_mode

import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
import os
import spectral

init_notebook_mode(connected=True)
%matplotlib inline

### ***Reading data***

In [ ]:
def loadData(name):
    
    data_path = os.path.join(os.getcwd(),'Data/')
   
    if name == 'SA1':
        
        data = sio.loadmat(os.path.join(data_path, 'SA1.mat'))['SA1']
        Train = sio.loadmat(os.path.join(data_path, 'SA1Train.mat'))['SA1Train']
        Test = sio.loadmat(os.path.join(data_path, 'SA1Test.mat'))['SA1Test']
        
    if name == 'SA2':
        
        data = sio.loadmat(os.path.join(data_path, 'SA2.mat'))['SA2']
        Train = sio.loadmat(os.path.join(data_path, 'SA2Train.mat'))['SA2Train']
        Test = sio.loadmat(os.path.join(data_path, 'SA2Test.mat'))['SA2Test']
        
    if name == 'SA3':
        
        data = sio.loadmat(os.path.join(data_path, 'SA3.mat'))['SA3']
        Train = sio.loadmat(os.path.join(data_path, 'SA3Train.mat'))['SA3Train']
        Test = sio.loadmat(os.path.join(data_path, 'SA3Test.mat'))['SA3Test']
    
    return data, Train, Test

In [ ]:
## GLOBAL VARIABLES
windowSize = 8

In [ ]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=testRatio, random_state=randomState,
                                                        stratify=y)
    return X_train, X_test, y_train, y_test

In [ ]:
def padWithZeros(X, margin=2):
    newX = np.zeros((X.shape[0] + 2 * margin, X.shape[1] + 2* margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

In [ ]:
def createImageCubes(X, y, windowSize=8, removeZeroLabels = True):
    margin = int((windowSize) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # split patches
    patchesData = np.zeros((X.shape[0] * X.shape[1], windowSize, windowSize, X.shape[2]))
    patchesLabels = np.zeros((X.shape[0] * X.shape[1]))
    patchIndex = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            patch = zeroPaddedX[r - margin:r + margin , c - margin:c + margin ]   
            patchesData[patchIndex, :, :, :] = patch
            patchesLabels[patchIndex] = y[r-margin, c-margin]
            patchIndex = patchIndex + 1
    if removeZeroLabels:
        patchesData = patchesData[patchesLabels>0,:,:,:]
        patchesLabels = patchesLabels[patchesLabels>0]
        patchesLabels -= 1
    return patchesData, patchesLabels

In [ ]:
dataset = 'SA1'
X1 , Train1, Test1 = loadData(dataset)
#X[X>100000]=-1
X1[np.isnan(X1)]=-1
X1[X1<-1000]=-1

In [ ]:
dataset = 'SA2'
X2 , Train2, Test2 = loadData(dataset)
#X[X>100000]=-1
X2[np.isnan(X2)]=-1
X2[X2<-1000]=-1

In [ ]:
dataset = 'SA3'
X3 , Train3, Test3 = loadData(dataset)
#X[X>100000]=-1
X3[np.isnan(X3)]=-1
X3[X3<-1000]=-1

In [ ]:
XTrain1, Train1 = createImageCubes(X1, Train1, windowSize=windowSize)
XTest1, Test1 = createImageCubes(X1, Test1, windowSize=windowSize)

XTrain1.shape,XTest1.shape, Train1.shape, Test1.shape

In [ ]:
XTrain2, Train2 = createImageCubes(X2, Train2, windowSize=windowSize)
XTest2, Test2 = createImageCubes(X2, Test2, windowSize=windowSize)

XTrain2.shape,XTest2.shape, Train2.shape, Test2.shape

In [ ]:
XTrain3, Train3 = createImageCubes(X3, Train3, windowSize=windowSize)
XTest3, Test3 = createImageCubes(X3, Test3, windowSize=windowSize)

XTrain3.shape,XTest3.shape, Train3.shape, Test3.shape

In [ ]:
trainS = np.concatenate((XTrain1, XTrain2, XTrain3) , axis = 0)
#trainS = trainS.reshape((trainS.shape[0],windowSize,windowSize,16,1))

testS = np.concatenate((XTest1, XTest2, XTest3) , axis = 0)
#testS = testS.reshape((testS.shape[0],windowSize,windowSize,16,1))

labelTr = np.concatenate((Train1, Train2, Train3) , axis = 0)
labelTs = np.concatenate((Test1, Test2, Test3) , axis = 0)


In [ ]:
test_ratio=0.9
trainS, testS2, labelTr, labelTs2 = splitTrainTestSet(trainS, labelTr, test_ratio)

In [ ]:
 np.min(labelTr), np.max(labelTr)

### ***Synthetic data generation with 3D GAN***

In [ ]:
# For running in python 2.x
from __future__ import print_function, unicode_literals
from __future__ import absolute_import, division
import numpy as np
import tensorflow as tf
from keras import backend as K
from keras.layers import Input, Dropout, Dense, RepeatVector, Lambda, Reshape, Conv3D, Conv2D, Flatten, InputSpec
from keras.layers import BatchNormalization, Concatenate, Multiply, Add, Conv2DTranspose, GlobalAveragePooling2D, MaxPool2D
from keras.layers.advanced_activations import LeakyReLU, Softmax
from keras.models import Model
from tensorflow.keras import layers

In [ ]:


def denseGamoGenCreate(latDim, num_class):
    noise = Input(shape=(latDim, ))
    labels = Input(shape=(num_class, ))
    gamoGenInput = Concatenate()([noise, labels])

    x = Dense(7 * 7 * 128, use_bias=False)(gamoGenInput)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU()(x)

    x = Reshape((7, 7, 128))(x)

    x = Conv2DTranspose(64, (5, 5), strides=(1, 1), padding='same', use_bias=False)(x)
    x = BatchNormalization(momentum=0.9)(x)
    x = LeakyReLU()(x)


    x = Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False)(x)
    x = BatchNormalization(momentum=0.9)(x)

    gamoGenFinal = Flatten()(x)

    gamoGen = Model([noise, labels], gamoGenFinal)
    return gamoGen



def denseGenProcessCreate(numMinor, dataMinor,sh,mul):
    ip1=Input(shape=(196,))
    x=Dense(numMinor, activation='softmax')(ip1)
    x=RepeatVector(mul)(x)
    z = np.reshape(dataMinor,(numMinor,mul))
    genProcessFinal=Lambda(lambda x: K.sum(x*K.transpose(K.constant(z)), axis=2))(x)
    genProcessReshape = Reshape(sh)(genProcessFinal)
    genProcess=Model(ip1, genProcessReshape)
    return genProcess

def denseDisCreate(sh, num_class):
   
    imIn=Input(shape=sh)

    conv_layer1 = Conv3D(filters=32, kernel_size=(1, 1, 7), activation='relu', padding='same')(imIn)
    conv_layer2 = Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu',padding='same')(conv_layer1)
    conv_layer2 = BatchNormalization()(conv_layer2)
    #conv_layer2 = MaxPool3D(pool_size=(1,1,3))(conv_layer2)
    
    conv3d_shape = conv_layer2.shape
    conv_layer2 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv_layer2)
    

    conv_layer3 = Conv2D(filters=64, kernel_size=(1,1), activation='relu',padding='same')(conv_layer2)
    conv_layer4 = Conv2D(filters=64, kernel_size=(3,3), activation='relu',padding='same')(conv_layer3)
    conv_layer4 = BatchNormalization()(conv_layer4)
    conv_layer4 = GlobalAveragePooling2D()(conv_layer4)
    
    flatten_layer = Flatten()(conv_layer4)
    
    
    labels=Input(shape=(num_class,))
    disInput=Concatenate()([flatten_layer, labels])
    x=Dropout(0.5)(disInput)
    
    disFinal1=Dense(20, activation='relu')(x)
    disFinal2=Dense(10, activation='relu')(disFinal1)
    disFinal=Dense(1, activation='sigmoid', kernel_initializer="he_normal")(disFinal2)
    
    dis=Model([imIn, labels], disFinal)
    return dis

def denseMlpCreate(sh, num_class):
    imIn=Input(shape=sh)

## convolutional layers
    conv_layer1 = Conv3D(filters=8, kernel_size=(3, 3, 7), activation='relu', padding='same')(imIn)
    conv_layer2 = Conv3D(filters=16, kernel_size=(3, 3, 5), activation='relu', padding='same')(conv_layer1)
    conv_layer3 = Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu', padding='same')(conv_layer2)
    conv3d_shape = conv_layer3.shape
    conv_layer3 = Reshape((conv3d_shape[1], conv3d_shape[2], conv3d_shape[3]*conv3d_shape[4]))(conv_layer3)
    conv_layer4 = Conv2D(filters=64, kernel_size=(3,3), activation='relu', padding='same')(conv_layer3)

    flatten_layer = Flatten()(conv_layer4)
    
    x=Dropout(0.5)(flatten_layer)
    
    mlpFinal1 = Dense(20, activation='relu')(x)
    mlpFinal2 = Dense(10, activation='relu')(mlpFinal1)
    mlpFinal = Dense(num_class, activation="softmax", kernel_initializer="he_normal")(mlpFinal2)
    
    mlp=Model(imIn, mlpFinal)
    return mlp

In [ ]:
# For running in python 2.x
from __future__ import print_function, unicode_literals
from __future__ import absolute_import, division

import sys
import numpy as np
from sklearn.metrics import confusion_matrix
from scipy.spatial.distance import cdist
from keras.utils.np_utils import to_categorical

def relabel(labelTr, labelTs):
    unqLab, pInClass=np.unique(labelTr, return_counts=True)
    sortedUnqLab=np.argsort(pInClass, kind='mergesort')
    c=sortedUnqLab.shape[0]
    labelsNewTr=np.zeros((labelTr.shape[0],))-1
    labelsNewTs=np.zeros((labelTs.shape[0],))-1
    pInClass=np.sort(pInClass)
    classMap=list()
    for i in range(c):
        labelsNewTr[labelTr==unqLab[sortedUnqLab[i]]]=i
        labelsNewTs[labelTs==unqLab[sortedUnqLab[i]]]=i
        classMap.append(np.where(labelsNewTr==i)[0])
    return labelsNewTr, labelsNewTs, c, pInClass, classMap, sortedUnqLab

def irFind(pInClass, c, irIgnore=1):
    ir=pInClass[-1]/pInClass
    imbalancedCls=np.arange(c)[ir>irIgnore]
    toBalance=np.subtract(pInClass[-1], pInClass[imbalancedCls])
    imbClsNum=toBalance.shape[0]
    if imbClsNum==0: sys.exit('No imbalanced classes found, exiting ...')
    return imbalancedCls, toBalance, imbClsNum, ir

def fileRead(fileName):
    dataTotal=np.loadtxt(fileName, delimiter=',')
    data=dataTotal[:, :-1]
    labels=dataTotal[:, -1]
    return data, labels

def indices(pLabel, tLabel):
    confMat=confusion_matrix(tLabel, pLabel)
    nc=np.sum(confMat, axis=1)
    tp=np.diagonal(confMat)
    tpr=tp/nc
    acsa=np.mean(tpr)
    gm=np.prod(tpr)**(1/confMat.shape[0])
    acc=np.sum(tp)/np.sum(nc)
    return acsa, gm, tpr, confMat, acc

def randomLabelGen(toBalance, batchSize, c):
    cumProb=np.cumsum(toBalance/np.sum(toBalance))
    bins=np.insert(cumProb, 0, 0)
    randomValue=np.random.rand(batchSize,)
    randLabel=np.digitize(randomValue, bins)-1
    randLabel_cat=to_categorical(randLabel)
    labelPadding=np.zeros((batchSize, c-randLabel_cat.shape[1]))
    randLabel_cat=np.hstack((randLabel_cat, labelPadding))
    return randLabel_cat

def batchDivision(n, batchSize):
    numBatches, residual=int(np.ceil(n/batchSize)), int(n%batchSize)
    if residual==0:
        residual=batchSize
    batchDiv=np.zeros((numBatches+1,1), dtype='int64')
    batchSizeStore=np.ones((numBatches, 1), dtype='int64')
    batchSizeStore[0:-1, 0]=batchSize
    batchSizeStore[-1, 0]=residual
    for i in range(numBatches):
        batchDiv[i]=i*batchSize
    batchDiv[numBatches]=batchDiv[numBatches-1]+residual
    return batchDiv, numBatches, batchSizeStore

def rearrange(labelsCat, numImbCls):
    labels=np.argmax(labelsCat, axis=1)
    arrangeMap=list()
    for i in range(numImbCls):
        arrangeMap.append(np.where(labels==i)[0])
    return arrangeMap

In [ ]:
from __future__ import print_function, unicode_literals
from __future__ import absolute_import, division

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from keras.preprocessing import image
from keras.layers import Input
from keras.models import Model
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

In [ ]:
data_path = 'Data'
fileName=['SA_trainData.csv', 'SA_testData.csv']
fileStart=data_path + 'SavedModel/'+'Salinas_GAMO'
fileEnd, savePath='_Model.h5', fileStart+'/'
adamOpt=Adam(0.0002, 0.5)
latDim, modelSamplePd, resSamplePd=100,1000, 500
plt.ion()

In [ ]:
batchSize, max_step=64,30000 #30000

#trainS, labelTr=spp.fileRead(fileName[0])
#testS, labelTs=spp.fileRead(fileName[1])

In [ ]:
labelTr, labelTs, c, pInClass, classMap, sortedUnqLab=relabel(labelTr, labelTs)


In [ ]:
imbalancedCls, toBalance, imbClsNum, ir=irFind(pInClass, c)

In [ ]:
labelsCat=to_categorical(labelTr)

In [ ]:
shuffleIndex=np.random.choice(np.arange(n), size=(n,), replace=False)
trainS=trainS[shuffleIndex]
labelTr=labelTr[shuffleIndex]
labelsCat=labelsCat[shuffleIndex]
classMap=list()
for i in range(c):
    classMap.append(np.where(labelTr==i)[0])

In [ ]:
# model initialization
sh=(windowSize,windowSize,16,1,)
mlp=denseMlpCreate(sh,8)
mlp.compile(loss='mean_squared_error', optimizer=adamOpt)
mlp.trainable=False

dis=denseDisCreate(sh,8)
dis.compile(loss='mean_squared_error', optimizer=adamOpt)
dis.trainable=False

In [ ]:
gen=denseGamoGenCreate(latDim,8)

In [ ]:
gen_processed, genP_mlp, genP_dis=list(), list(), list()
for i in range(imbClsNum):
    dataMinor=trainS[classMap[i], :]
    numMinor=dataMinor.shape[0]
    print(dataMinor.shape)
    print(numMinor)
    gen_processed.append(denseGenProcessCreate(numMinor, dataMinor,sh = (windowSize,windowSize,16,1),mul = windowSize*windowSize*16 ))

    ip1=Input(shape=(latDim,))
    ip2=Input(shape=(c,))
    op1=gen([ip1, ip2])
    op2=gen_processed[i](op1)
    op3=mlp(op2)
    genP_mlp.append(Model(inputs=[ip1, ip2], outputs=op3))
    genP_mlp[i].compile(loss='mean_squared_error', optimizer=adamOpt)

    ip1=Input(shape=(latDim,))
    ip2=Input(shape=(c,))
    ip3=Input(shape=(c,))
    op1=gen([ip1, ip2])
    op2=gen_processed[i](op1)
    op3=dis([op2, ip3])
    genP_dis.append(Model(inputs=[ip1, ip2, ip3], outputs=op3))
    genP_dis[i].compile(loss='mean_squared_error', optimizer=adamOpt)

In [ ]:
batchDiv, numBatches, bSStore=batchDivision(n, batchSize)
genClassPoints=int(np.ceil(batchSize/c))
#fig, axs=plt.subplots(imbClsNum, 3)

In [ ]:
if not os.path.exists(fileStart):
    os.makedirs(fileStart)
picPath=savePath+'Pictures'
if not os.path.exists(picPath):
    os.makedirs(picPath)

In [ ]:
iter=int(np.ceil(max_step/resSamplePd)+1)
acsaSaveTr, gmSaveTr, accSaveTr=np.zeros((iter,)), np.zeros((iter,)), np.zeros((iter,))
acsaSaveTs, gmSaveTs, accSaveTs=np.zeros((iter,)), np.zeros((iter,)), np.zeros((iter,))
confMatSaveTr, confMatSaveTs=np.zeros((iter, c, c)), np.zeros((iter, c, c))
tprSaveTr, tprSaveTs=np.zeros((iter, c)), np.zeros((iter, c))

In [ ]:
step=0
while step<max_step:
    for j in range(numBatches):
        x1, x2=batchDiv[j, 0], batchDiv[j+1, 0]
        validR=np.ones((bSStore[j, 0],1))-np.random.uniform(0,0.1, size=(bSStore[j, 0], 1))
        mlp.train_on_batch(trainS[x1:x2], labelsCat[x1:x2])
        dis.train_on_batch([trainS[x1:x2], labelsCat[x1:x2]], validR)

        invalid=np.zeros((bSStore[j, 0], 1))+np.random.uniform(0, 0.1, size=(bSStore[j, 0], 1))
        randNoise=np.random.normal(0, 1, (bSStore[j, 0], latDim))
        fakeLabel=randomLabelGen(toBalance, bSStore[j, 0], c)
        rLPerClass=rearrange(fakeLabel, imbClsNum)
        fakePoints=np.zeros((bSStore[j, 0],windowSize,windowSize,16,1))
        genFinal=gen.predict([randNoise, fakeLabel])
        for i1 in range(imbClsNum):
            if rLPerClass[i1].shape[0]!=0:
                temp=genFinal[rLPerClass[i1]]
                fakePoints[rLPerClass[i1]]=gen_processed[i1].predict(temp)

        mlp.train_on_batch(fakePoints, fakeLabel)
        dis.train_on_batch([fakePoints, fakeLabel], invalid)

        for i1 in range(imbClsNum):
            validA=np.ones((genClassPoints, 1))
            randomLabel=np.zeros((genClassPoints, c))
            randomLabel[:, i1]=1
            randNoise=np.random.normal(0, 1, (genClassPoints, latDim))
            oppositeLabel=np.ones((genClassPoints, c))-randomLabel
            genP_mlp[i1].train_on_batch([randNoise, randomLabel], oppositeLabel)
            genP_dis[i1].train_on_batch([randNoise, randomLabel, randomLabel], validA)

        if step%resSamplePd==0:
            saveStep=int(step//resSamplePd)

            pLabel=np.argmax(mlp.predict(trainS), axis=1)
            acsa, gm, tpr, confMat, acc=indices(pLabel, labelTr)
            print('Train: Step: ', step, 'ACSA: ', np.round(acsa, 4), 'GM: ', np.round(gm, 4))
            print('TPR: ', np.round(tpr, 2))
            acsaSaveTr[saveStep], gmSaveTr[saveStep], accSaveTr[saveStep]=acsa, gm, acc
            confMatSaveTr[saveStep]=confMat
            tprSaveTr[saveStep]=tpr

            pLabel=np.argmax(mlp.predict(testS), axis=1)
            acsa, gm, tpr, confMat, acc=indices(pLabel, labelTs)
            print('Test: Step: ', step, 'ACSA: ', np.round(acsa, 4), 'GM: ', np.round(gm, 4))
            print('TPR: ', np.round(tpr, 2))
            acsaSaveTs[saveStep], gmSaveTs[saveStep], accSaveTs[saveStep]=acsa, gm, acc
            confMatSaveTs[saveStep]=confMat
            tprSaveTs[saveStep]=tpr


        if step%modelSamplePd==0 and step!=0:
            direcPath=savePath+'gamo_models_'+str(step)
            if not os.path.exists(direcPath):
                os.makedirs(direcPath)
            gen.save(direcPath+'/GEN_'+str(step)+fileEnd)
            mlp.save(direcPath+'/MLP_'+str(step)+fileEnd)
            dis.save(direcPath+'/DIS_'+str(step)+fileEnd)
            for i in range(imbClsNum):
                gen_processed[i].save(direcPath+'/GenP_'+str(i)+'_'+str(step)+fileEnd)

        step=step+2
        if step>=max_step: break

In [ ]:
pLabel=np.argmax(mlp.predict(testS), axis=1)
acsa, gm, tpr, confMat, acc=indices(pLabel, labelTs)
print('Performance on Test Set: Step: ', step, 'ACSA: ', np.round(acsa, 4), 'GM: ', np.round(gm, 4))
print('TPR: ', np.round(tpr, 2))
acsaSaveTs[-1], gmSaveTs[-1], accSaveTs[-1]=acsa, gm, acc
confMatSaveTs[-1]=confMat
tprSaveTs[-1]=tpr

In [ ]:
# Classification Report
print(classification_report(labelTs, (pLabel),
      target_names = [
"Bog",
"Water",
"Marsh",
"Crop",
"Forested Wetland",
"Shrub Wetland",
"Urban",
"Fen"


                      ]))

In [ ]:
ca = np.sum(pLabel == labelTs) / labelTs.shape[0]

print("Classification accuracy: %.5f" % ca)

In [ ]:
direcPath=savePath+'gamo_models_'+str(step)
if not os.path.exists(direcPath):
    os.makedirs(direcPath)
gen.save(direcPath+'/GEN_'+str(step)+fileEnd)
mlp.save(direcPath+'/MLP_'+str(step)+fileEnd)
dis.save(direcPath+'/DIS_'+str(step)+fileEnd)
for i in range(imbClsNum):
    gen_processed[i].save(direcPath+'/GenP_'+str(i)+'_'+str(step)+fileEnd)

In [ ]:
resSave=savePath+'Results.txt'
np.savez(resSave, acsa=acsa, gm=gm, tpr=tpr, confMat=confMat, acc=acc)
recordSave=savePath+'Record.txt'
np.savez(recordSave, acsaSaveTr=acsaSaveTr, gmSaveTr=gmSaveTr, accSaveTr=accSaveTr, acsaSaveTs=acsaSaveTs, gmSaveTs=gmSaveTs, accSaveTs=accSaveTs, confMatSaveTr=confMatSaveTr, confMatSaveTs=confMatSaveTs, tprSaveTr=tprSaveTr, tprSaveTs=tprSaveTs)

In [ ]:
X = np.concatenate((trainS , testS) , axis = 0)
print(X.shape)
Y = np.concatenate((labelTr , labelTs) , axis = 0)
Y.shape

In [ ]:
unqLab, pInClass=np.unique(labelTr, return_counts=True)
print(unqLab,pInClass)
pInClass = 1792 - pInClass
print(pInClass)

In [ ]:
#y = []
for i1 in range(pInClass.shape[0] - 1):
  testNoise=np.random.normal(0, 1, (pInClass[i1], latDim))
  testLabel=np.zeros((pInClass[i1], c))
  testLabel[:, i1]=1
  genFinal=gen.predict([testNoise, testLabel])
  genImages=gen_processed[i1].predict(genFinal)
  genImages=np.reshape(genImages, (pInClass[i1], 8,8,16,1))
  X =  np.concatenate((X , genImages) , axis = 0)
  Y =  np.concatenate((Y , np.argmax(testLabel, axis=1)) , axis = 0)
print(X.shape)
print(np.array(Y).shape)

In [ ]:
import scipy.io as sio
sio.savemat('Data/X_Avalon.mat', {'X':X})
sio.savemat('Data/Y_Avalon.mat', {'Y':Y})

### ***Vision Transformer***

In [ ]:
################################           Reading mat file

X=sio.loadmat('Data/X_Avalon.mat')
Y=sio.loadmat('Data/Y_Avalon.mat')
X = X['X']
Y = Y['Y']
Y=Y.reshape(Y[0].shape)
Y.shape, X.shape

In [ ]:
windowSize=8

In [ ]:
X = X.reshape((X.shape[0],windowSize,windowSize,16))
X.shape

In [ ]:
Xtrain= X
ytrain= Y

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_addons as tfa

In [ ]:
num_classes = 8
input_shape = (8, 8, 16)

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 256
num_epochs = 100
image_size = 8  # We'll resize input images to this size
patch_size = 2  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 32
num_heads = 8
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 8
mlp_head_units = [64, 128]  

In [ ]:
data_augmentation = keras.Sequential(
    [
        layers.Normalization(),
        layers.Resizing(image_size, image_size),
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(factor=0.02),
        layers.RandomZoom(
            height_factor=0.2, width_factor=0.2
        ),
    ],
    name="data_augmentation",
)
# Compute the mean and the variance of the training data for normalization.
data_augmentation.layers[0].adapt(Xtrain)

In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

In [ ]:
def create_vit_classifier():
   
    inputs = layers.Input(shape=input_shape)
    
############################################### ViT classifier

# Compute the mean and the variance of the training data for normalization.
   # data_augmentation.layers[0].adapt(x)
    # Augment data.
    augmented = data_augmentation(inputs)
    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model

In [ ]:
model = create_vit_classifier()

In [ ]:

optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

model.compile(
        optimizer=optimizer,
        loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.SparseCategoricalAccuracy(name="accuracy"),
            keras.metrics.SparseTopKCategoricalAccuracy(5, name="top-5-accuracy"),
        ],
    )

checkpoint_filepath = "ViT/checkpoint"
checkpoint_callback = keras.callbacks.ModelCheckpoint(
        checkpoint_filepath,
        monitor="val_accuracy",
        save_best_only=True,
        save_weights_only=True,
    )

history = model.fit(
        x=Xtrain,
        y=ytrain,
        batch_size=batch_size,
        epochs=100,
        validation_split=0.1,
    
    )
